In [1]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
import numpy as np
import cv2
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd
from shutil import copyfile
from os import getcwd


In [2]:
main_dir = r'C:\Users\joaom\OneDrive\Desktop\mask_or_nomask'
mask_dir = r'C:\Users\joaom\OneDrive\Desktop\mask_or_nomask\maskd'
no_mask_dir =r'C:\Users\joaom\OneDrive\Desktop\mask_or_nomask\nomaskd'
test = r'C:\Users\joaom\OneDrive\Desktop\mask_or_nomask\test'
train = r'C:\Users\joaom\OneDrive\Desktop\mask_or_nomask\train'
train_mask = r'C:\Users\joaom\OneDrive\Desktop\mask_or_nomask\train\train_mask'
train_no_mask =r'C:\Users\joaom\OneDrive\Desktop\mask_or_nomask\train\train_no_mask'
test_mask = r'C:\Users\joaom\OneDrive\Desktop\mask_or_nomask\test\test_mask'
test_no_mask = r'C:\Users\joaom\OneDrive\Desktop\mask_or_nomask\test\test_no_mask'
 
print(len(os.listdir(main_dir)))
print(len(os.listdir(mask_dir)))
print(len(os.listdir(no_mask_dir)))

4
950
1000


In [3]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    source_file_list = random.sample(os.listdir(SOURCE), len(os.listdir(SOURCE)))
    c=0
    while c < len(source_file_list):
        file = source_file_list[c]
        if os.path.getsize(os.path.join(SOURCE,file)) > 0:
            file_being_sent = os.path.join(SOURCE,file)
            if len(os.listdir(TRAINING)) < len(source_file_list)*SPLIT_SIZE:
                copyfile(file_being_sent,os.path.join(TRAINING,file))
                b=1
            else:
                copyfile(file_being_sent,os.path.join(TESTING,file))
        c+=1
        
split_size = .9
split_data(mask_dir, train_mask, test_mask, split_size)
split_data(no_mask_dir, train_no_mask, test_no_mask, split_size)

In [4]:
print(len(os.listdir(train_mask)))
print(len(os.listdir(test_mask)))
print(len(os.listdir(train_no_mask)))
print(len(os.listdir(test_no_mask)))

855
95
900
100


In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

In [6]:
TRAINING_DIR = r'C:\Users\joaom\OneDrive\Desktop\mask_or_nomask\train'
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# NOTE: YOU MUST USE A BATCH SIZE OF 10 (batch_size=10) FOR THE 
# TRAIN GENERATOR.
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=10,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

VALIDATION_DIR = r'C:\Users\joaom\OneDrive\Desktop\mask_or_nomask\test'
validation_datagen = ImageDataGenerator(rescale = 1./255)

# NOTE: YOU MUST USE A BACTH SIZE OF 10 (batch_size=10) FOR THE 
# VALIDATION GENERATOR.
validation_generator = train_datagen.flow_from_directory(VALIDATION_DIR,
                                                    batch_size=10,
                                                    class_mode='binary',
                                                    target_size=(150, 150)) 



# Expected Output:
# Found 2700 images belonging to 2 classes.
# Found 300 images belonging to 2 classes.

Found 1755 images belonging to 2 classes.
Found 195 images belonging to 2 classes.


In [7]:
history = model.fit_generator(train_generator,
                              epochs=1,
                              verbose=1,
                              validation_data=validation_generator)

IndentationError: unexpected indent (<ipython-input-7-445aef2f8ba7>, line 2)

In [8]:
import numpy as np
# import google.colab
# from google.colab import files
from keras.preprocessing import image

test = r'C:\Users\joaom\OneDrive\Desktop\fjftyt\01000'
list1 = os.listdir(test)

for value in list1[:10]:
    img = image.load_img(os.path.join(test,value), target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    if str(classes) == '[[1.]]':
        print('No Mask','/',str(value)[5:])
    else : 
        print('Mask','/',str(value)[5:])
    


Using TensorFlow backend.


Mask / .png
Mask / .png
Mask / _Mask.jpg
Mask / .png
Mask / _Mask.jpg
Mask / .png
Mask / _Mask.jpg
Mask / .png
Mask / _Mask.jpg
Mask / .png


In [29]:
cap = cv2.VideoCapture(0)
while True:
    ret, image_np = cap.read()
    img = cv2.resize(image_np,(150, 150))
    img = np.expand_dims(img, axis=0)
    text = model.predict(img, batch_size=10)
    if str(text) == '[[1.]]':
        #print('No Mask','/',str(value)[5:])
        r = 'No Mask'
    else : 
        r= 'Mask'
#         print('Mask','/',str(value)[5:])
    
    image_np = cv2.putText(image_np,f"{r}, {text}",(50, 100),cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 255),2) 
    cv2.imshow('mask detection',cv2.resize(image_np,(800,600)))
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break